In [1]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
import matplotlib.pyplot as plt
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
import sys
import os
import syspend
from utility import parse_config, seed_everything, custom_print
from preprocess_class import create_datasets
from model_base_class import BaseModel
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from transformers.pipelines import pipeline
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired, ZeroShotClassification
from BERTopic_model import BERTopic_model
import plotly

sys.path.append("../..")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tingy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tingy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tingy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tingy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('../../data/reviews.csv')
train, test = create_datasets(df)

In [56]:
model = BERTopic_model()
model.topic_model = BERTopic.load('sentence-transformers-key-bigram')

In [49]:
miniLM_model.topic_model.save("sentence-transformers-key-bigram")	


In [123]:
topic_model.get_topic_info()

,Topic,Count,Name,CustomName
0,-1,902,-1_pork rinds_microwave pork_vanilla beans_raw...,Outliers
1,0,1406,0_tangerine flavor_tangerine orange_orange tan...,Soft Drinks
2,1,624,1_quality tea_quality coffee_recommend tea_tea...,Tea/Coffee
3,2,167,2_dog foods_dog treats_products dogs_dogs chicken,Dog Food
4,3,163,3_sauce tasty_ranchero sauce_fajita sauce_spic...,Sauces
5,4,156,4_bisquick pancakes_cake mixes_pancake mixes_b...,Baking Products
6,5,111,5_diet bars_tasting bars_ingredients bars_tast...,Snack Bars
7,6,110,6_chips flavor_chips tasty_tasting chips_love ...,Chips
8,7,100,7_keurig coffee_kona coffee_keurig nice_keurig...,Keurig Coffee Products
9,8,91,8_healthy cereals_cereal tasty_cereal snack_ta...,Cereal


In [55]:
topic_model.get_topic(1)

[('the', 0.050298565034729265),
 ('and', 0.04202089928639583),
 ('it', 0.04066654586185052),
 ('to', 0.03776439526073845),
 ('this', 0.03678709696743862),
 ('is', 0.031280962615838605),
 ('of', 0.030870238957200563),
 ('for', 0.02765312690854749),
 ('in', 0.025100899786985174),
 ('was', 0.024953398287246218)]

In [52]:
labeled_train = pd.DataFrame({'Text': train.text,'Label':topic_model.topics_})
labeled_train.loc[labeled_train['Label']==4, 'Text'].sample(n=10)

772     I purchased this at my local health food store...
820     I realize that it is not quite healthy to guzz...
1799    I purchased this lime juice a few years ago on...
2123    This is a good drink, tastes like soda, but fr...
2302    This is for sure not what I was expecting it t...
3848    I've tried other flavors of Switch carbonated ...
1781    I expected to like this drink because I like v...
1719    I like orange soda but usually cannot drink an...
3868    So far I like this cherry juice best.  I was e...
3142    I wasn't sure what to expect with this citrusy...
Name: Text, dtype: object

In [98]:
model.topic_model.get_topic_freq().shape[0] -1

8

In [47]:
miniLM_model.topic_model.set_topic_labels({
-1: 'Outliers',
0: 'Soft Drinks',
1: 'Tea/Coffee',
2: 'Dog Food',
3: 'Sauces',
4: 'Baking Products',
5: 'Snack Bars',
6: 'Chips',
7: 'Keurig Coffee Products',
8: 'Cereal',
9: 'Noodle',
10: 'Salty Food Products',
11: 'Crackers',
12: 'Cat Food',
13: 'Coconut Products',
14: 'Popcorn',
15: 'Oil Products',
16: 'Soup',
17: 'Protein Powder',
18: 'Peanut Butter'
})

In [46]:
labeled_train = pd.DataFrame({'Text': train.text,'Label':miniLM_model.topic_model.topics_})
labeled_train.loc[labeled_train['Label']==7, 'Text'].sample(n=10)

1530    this product is hard to find for us in stores ...
2138    I was expecting a plain old cup of coffee when...
1323    The products lid stays attached to the needle ...
937     I love Sumatra coffee and was excited to recei...
2797    Although I've only been able to try three of C...
2338    I love these K cups they have a nice tea taste...
1407    I'm a pretty big coffee drinker, but had never...
2409    I enjoyed these k-cups when I used them. They ...
676     I have tried about 25 different K-Cup flavors,...
139     Caused my cuisinart k-cup brewer to leak every...
Name: Text, dtype: object

In [68]:
miniLM_model.topic_model.set_topic_labels({
-1: 'Outliers',
0: 'Staple Food/Cooking Products',
1: 'Coffee',
2: 'Tea',
3: 'Snacks',
4: 'Dog Food',
5: 'Baby Food/Canned Food',
6: 'Delivery',
7: 'Processed Food',
8: 'Cat Food',
9: 'Concentrated Syrup',
10: 'Popcorn',
11: 'Coconut Products',
12: 'Protein Powder',
13: 'Consumable Oil',
14: 'Soup'})

In [83]:
labeled_test.groupby('Label').count()

,Text,Topic,CustomName
Label,,,
-1,400,400,400
0,262,262,262
1,107,107,107
2,107,107,107
3,67,67,67
4,55,55,55
5,41,41,41
6,37,37,37
7,13,13,13


In [99]:
labeled_test.drop(columns=['Label'], inplace=True)
labeled_test.to_csv(f'{model_name}.csv', index=False)

,Text,Label,Topic,CustomName
0,"This tea is amazing, I cannot believe the pric...",0,0,Drinks
1,I've tried a handful of other brands of French...,0,0,Drinks
2,I would recommend not buying these nuts if you...,2,2,Healthy Alternatives
3,"I wish these came in larger quantities, if I c...",-1,-1,Outliers
4,Has a smooth taste like it claims without any ...,0,0,Drinks
...,...,...,...,...
1084,I'd seen an add for these in a magazine recent...,-1,-1,Outliers
1085,I was a bit cautious when I first opened the b...,-1,-1,Outliers
1086,"I bought these for my wife for her birthday, s...",-1,-1,Outliers
1087,Lowrey's has changed it's microwave pork rinds...,-1,-1,Outliers


In [65]:
topics_to_merge= [[0,1,7,12,13,21,28], [2,14,22,8,20], [3,25,31],[5,23],[9,10,16,19],[6,11,15,24,26,17],
                  [-1,27,29,30]]
model.topic_model.merge_topics(train.text, topics_to_merge)

In [80]:
model.topic_model.get_topic_info()

,Topic,Count,Name,CustomName
0,-1,1406,-1_rinds_pork rinds_lentils_similac,Outliers
1,0,972,0_drinks___,Drinks
2,1,563,1_chicken jerky_movie_products china_theater,Snacks
3,2,452,2_healthy alternatives___,Healthy Alternatives
4,3,340,3_carbohydrates___,Carbohydrates
5,4,244,4_household___,Household
6,5,174,5_sauce___,Sauce
7,6,146,6_dog___,Dogs
8,7,58,7_cat___,Cats


In [79]:
model.topic_model.set_topic_labels({-1: "Outliers", 0: "Drinks", 1: "Snacks", 2:"Healthy Alternatives",
                                    3:"Carbohydrates", 4:"Household", 5:"Sauce", 6:'Dogs', 7:"Cats"})

In [58]:
topics_over_time = model.topic_model.topics_over_time(docs=train.text, 
                                                timestamps=train.date, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)
model.topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=15)

19it [02:44,  8.67s/it]


In [88]:
# Save model
model.topic_model.save("zero-shot")	

In [111]:
# Load model
topic_model = BERTopic.load("sentence-transformers-key-bigram-v2")	

In [116]:
df = pd.DataFrame({'Text':train.text, 'Topic':topic_model.topics_, 'Date': train.date, 'Sentiment':train.sentiments})

In [128]:
df = df.merge(topic_model.get_topic_info().loc[:,['Topic','CustomName']])

In [131]:
df = df[['Text','Topic','CustomName', 'Date', 'Sentiment']]

Visualisation

In [26]:
sent_df = pd.read_csv('sentence-transformers-key-bigram-v2_full.csv')
sent_df['Date'] = test.date
sent_df['Sentiment'] = test.sentiments

In [134]:
sent_df.loc[sent_df['Sentiment'] == 'positive', 'Sentiment'] = 1
sent_df.loc[sent_df['Sentiment'] == 'negative', 'Sentiment'] = 0


In [133]:
sent_df = sent_df.append(df)

In [135]:
sent_df

,Text,Topic,CustomName,Date,Sentiment
0,"This tea is amazing, I cannot believe the pric...",1,Tea/Coffee,2021-08-06,1
1,I've tried a handful of other brands of French...,1,Tea/Coffee,2021-04-09,1
2,I would recommend not buying these nuts if you...,0,Soft Drinks,2020-09-12,0
3,"I wish these came in larger quantities, if I c...",0,Soft Drinks,2021-04-21,1
4,Has a smooth taste like it claims without any ...,1,Tea/Coffee,2020-01-13,1
...,...,...,...,...,...
4350,"add to aniything!! soup, salad, meat... brings...",16,Soup,2021-11-03,1
4351,"This makes a good, quick and sustaining meal w...",16,Soup,2019-12-18,1
4352,"The taste is good, but I found myself biting d...",16,Soup,2020-07-03,0
4353,I sent this case of soup to my son. Great deal...,16,Soup,2019-08-11,1


In [136]:
group_df_time =sent_df.groupby(['Topic', pd.Grouper(freq="6M",key='Date')]).agg(Positive=('Sentiment', np.sum), Count = ('Sentiment', len), Avg=('Sentiment', lambda x: x.sum()/len(x)))
#Count positive and negative sentiment, positive/total, use this value to see how it changes

In [137]:
group_df_time.reset_index(inplace=True)

In [142]:
group_df_time

,Topic,Date,Positive,Count,Avg
0,-1,2017-07-31,2,4,0.500000
1,-1,2018-01-31,7,11,0.636364
2,-1,2018-07-31,22,31,0.709677
3,-1,2019-01-31,29,45,0.644444
4,-1,2019-07-31,24,35,0.685714
...,...,...,...,...,...
165,18,2019-07-31,2,3,0.666667
166,18,2020-07-31,7,8,0.875000
167,18,2021-01-31,7,7,1.000000
168,18,2021-07-31,13,14,0.928571


In [138]:
pivot_df_time = group_df_time.pivot(index='Date',columns='Topic', values='Avg')

In [139]:
pivot_df_time

Topic,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Date,,,,,,,,,,,,,,,,,,,,
2017-07-31,0.500000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-31,0.636364,0.680000,0.500000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,1.000000,NaN,0.666667,0.500000,1.000000,1.000000
2018-07-31,0.709677,0.787879,0.888889,1.000000,1.000000,0.833333,1.000000,0.666667,0.500000,1.000000,1.000000,0.250000,1.000000,0.000000,1.000000,1.000000,1.000000,NaN,1.000000,NaN
2019-01-31,0.644444,0.711864,0.857143,1.000000,1.000000,0.714286,0.714286,0.571429,1.000000,1.000000,1.000000,0.500000,1.000000,NaN,NaN,0.500000,NaN,NaN,1.000000,1.000000
2019-07-31,0.685714,0.693878,0.846154,1.000000,1.000000,0.500000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.800000,1.000000,NaN,0.500000,NaN,0.666667
2020-01-31,0.712644,0.705882,0.776119,0.583333,0.866667,0.800000,0.894737,1.000000,0.857143,0.800000,1.000000,0.800000,0.666667,0.666667,0.333333,1.000000,0.833333,0.571429,1.000000,NaN
2020-07-31,0.725806,0.721311,0.833333,0.692308,0.952381,0.869565,0.952381,0.625000,0.583333,1.000000,1.000000,0.916667,0.428571,0.800000,0.583333,0.600000,1.000000,0.375000,1.000000,0.875000
2021-01-31,0.724138,0.708487,0.846939,0.259259,0.857143,0.724138,0.913043,0.333333,0.777778,0.923077,1.000000,0.750000,0.882353,0.363636,0.272727,0.777778,0.800000,0.700000,1.000000,1.000000
2021-07-31,0.700980,0.732888,0.792517,0.517647,0.888889,0.604938,0.833333,0.647887,0.650794,0.953488,0.918919,0.821429,1.000000,0.551724,0.733333,0.900000,0.750000,0.583333,0.785714,0.928571


In [144]:
group_df =sent_df.groupby(['CustomName'], as_index=False, group_keys=False).agg(Positive=('Sentiment', np.sum), Count = ('Sentiment', len), Avg=('Sentiment', lambda x: x.sum()/len(x)))


In [145]:
fig = px.line(pivot_df_time, x=pivot_df_time.index, y =pivot_df_time.columns)
fig.show()

In [143]:
group_df_time.to_csv('out.csv')

In [146]:
import plotly.express as px
fig = px.bar(group_df, x='CustomName', y ='Avg')
fig.show()